### Import tf.keras
`tf.keras` is TensorFlow's implementation of the `Keras API specification`. This is a high-level API to build and train models that includes first-class support for TensorFlow-specific functionality, such as `eager execution`, `tf.data` pipelines, and `Estimators`. `tf.keras` makes TensorFlow easier to use without sacrificing flexibility and performance.

To get started, import `tf.keras` as part of your TensorFlow program setup:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow import keras

`tf.keras` can run any Keras-compatible code, but keep in mind:

The `tf.keras` version in the latest TensorFlow release might not be the same as the latest keras version from PyPI. Check `tf.keras.version`.
When `saving a model's weights`, `tf.keras` defaults to the `checkpoint format`. Pass `save_format='h5'` to use HDF5 (or pass a filename that ends in .h5).

In [5]:
tf.keras.__version__, tf.__version__

('2.2.4-tf', '2.1.0')

### Build a simple model
#### Sequential model
In Keras, you assemble *lyaers* to build *models*. A model is (usually) a graph of lyaers. The most common type of model is a stack of layers: `tf.keras.Sequential` model.

To build a simple, fully-connected network (i.e. multi-layer perceptron):

In [6]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

#### Configure the layers
There are many `tf.keras.layers` available. Most of them share some common constructor arguments:

- `activation`: Set the activation function for the layer. This parameter is specified by the name of a built-in function or as a callable object. By default, no activation is applied.
- `kernel_initializer` and `bias_initializer`: The initialization schemes that create the layer's weights (kernel and bias). This parameter is a name or a callable object. This defaults to the `"Glorot uniform"` initializer.
- `kernel_regularizer` and `bias_regularizer`: The regularization schemes that apply the layer's weights (kernel and bias), such as L1 or L2 regularization. By default, no regularization is applied.

The following instantiates tf.keras.layers.Dense layers using constructor arguments:

In [9]:
# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

### Train and evaluate
#### Set up training
After the model is constructed, configure its learning process by calling the `compile` method:

In [32]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(63, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

`tf.keras.Model.compile` takes three important arguments:

- `optimizer`: This object specifies the training procedure. Pass it optimizer instances from the `tf.keras.optimizers` module, such as `tf.keras.optimizers.Adam` or `tf.keras.optimizers.SGD`. If you just want to use the default parameters, you can also specify optimizers via strings, such as `'adam'` or `'sgd'`.
- `loss`: The function to minimize during optimization. Common choices include mean square error (`mse`), `categorical_crossentropy`, and `binary_crossentropy`. Loss functions are specified by name or by passing a callable object from the `tf.keras.losses` module.
- `metrics`: Used to monitor training. These are string names or callables from the `tf.keras.metrics` module.
Additionally, to make sure the model trains and evaluates eagerly, you can make sure to pass `run_eagerly=True` as a parameter to compile.

In [29]:
dir(tf.keras.losses)

['BinaryCrossentropy',
 'CategoricalCrossentropy',
 'CategoricalHinge',
 'CosineSimilarity',
 'Hinge',
 'Huber',
 'KLD',
 'KLDivergence',
 'LogCosh',
 'Loss',
 'MAE',
 'MAPE',
 'MSE',
 'MSLE',
 'MeanAbsoluteError',
 'MeanAbsolutePercentageError',
 'MeanSquaredError',
 'MeanSquaredLogarithmicError',
 'Poisson',
 'Reduction',
 'SparseCategoricalCrossentropy',
 'SquaredHinge',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'binary_crossentropy',
 'categorical_crossentropy',
 'categorical_hinge',
 'cosine_similarity',
 'deserialize',
 'get',
 'hinge',
 'kld',
 'kullback_leibler_divergence',
 'logcosh',
 'mae',
 'mape',
 'mean_absolute_error',
 'mean_absolute_percentage_error',
 'mean_squared_error',
 'mean_squared_logarithmic_error',
 'mse',
 'msle',
 'poisson',
 'serialize',
 'sparse_categorical_crossentropy',
 'squared_hinge']

The following shows a few examples of configuring a model for training:

In [33]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',
              metrics=['mae'])

# Configure a model for categorical classification.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

#### Train from NumPy data
For small datasets, use in-memory `NumPy` arrays to train and evaluate a model. The model is "fit" to the training data using `fit` method:

In [34]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 212us/sample - loss: 39.2018 - categorical_accuracy: 0.0980
Epoch 2/10
1000/1000 [==============================] - 0s 47us/sample - loss: 296.0919 - categorical_accuracy: 0.0960
Epoch 3/10
1000/1000 [==============================] - 0s 72us/sample - loss: 1095.4612 - categorical_accuracy: 0.1110
Epoch 4/10
1000/1000 [==============================] - 0s 76us/sample - loss: 2757.6446 - categorical_accuracy: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 75us/sample - loss: 6179.7515 - categorical_accuracy: 0.0980
Epoch 6/10
1000/1000 [==============================] - 0s 75us/sample - loss: 12462.6728 - categorical_accuracy: 0.1000
Epoch 7/10
1000/1000 [==============================] - 0s 83us/sample - loss: 17707.5759 - categorical_accuracy: 0.0840
Epoch 8/10
1000/1000 [==============================] - 0s 81us/sample - loss: 26576.2881 - categorical_accuracy: 0.1040
Epoch 9/10
1000/1

`tf.keras.Model.fit` takes three important arguments:

- `epochs`: Training is structured into epochs. An epoch is one iteration over the entire input data (this is done in smaller batches).
- `batch_size`: When passed NumPy data, the model slices the data into smaller batches and iterates over these batches during training. This integer specifies the size of each batch. Be aware that the last batch may be smaller if the total number of samples is not divisible by the batch size.
- `validation_data`: When prototyping a model, you want to easily monitor its performance on some validation data. Passing this argument—a tuple of inputs and labels—allows the model to display the loss and metrics in inference mode for the passed data, at the end of each epoch.

Here's an example using `validation_data`:

In [36]:
val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 117us/sample - loss: 51166.4235 - categorical_accuracy: 0.1030 - val_loss: 32734.4218 - val_categorical_accuracy: 0.1100
Epoch 2/10
1000/1000 [==============================] - 0s 59us/sample - loss: 40367.2308 - categorical_accuracy: 0.0980 - val_loss: 39302.3569 - val_categorical_accuracy: 0.1200
Epoch 3/10
1000/1000 [==============================] - 0s 62us/sample - loss: 55623.4503 - categorical_accuracy: 0.1220 - val_loss: 47860.7773 - val_categorical_accuracy: 0.1100
Epoch 4/10
1000/1000 [==============================] - 0s 98us/sample - loss: 60513.1189 - categorical_accuracy: 0.0930 - val_loss: 79568.2546 - val_categorical_accuracy: 0.1200
Epoch 5/10
1000/1000 [==============================] - 0s 124us/sample - loss: 88207.1187 - categorical_accuracy: 0.1070 - val_loss: 84485.8104 - val_categorical_accuracy: 0.1200
Epoch 6/10
1000/1000 [==============================] - 

#### Train from tf.data datasets
Use the `Dataset API` to scale to large datasets or multi-device training. Pass a `tf.data.Dataset` instance to the `fit` method:

In [37]:
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.fit(dataset, epochs=10)

Train for 32 steps
Epoch 1/10
32/32 [==============================] - 0s 5ms/step - loss: 199476.3549 - categorical_accuracy: 0.1000
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 170337.5194 - categorical_accuracy: 0.0960
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 173876.1374 - categorical_accuracy: 0.1100
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 220853.5948 - categorical_accuracy: 0.1130
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 244880.4685 - categorical_accuracy: 0.1060
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 255554.5268 - categorical_accuracy: 0.0800
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 289361.2976 - categorical_accuracy: 0.0920
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 297669.1155 - categorical_accuracy: 0.1010
Epoch 9/10
32/32 [==============================] - 0s 4ms/st

Since the `Dataset` yields batches of data, this snippet does not require a `batch_size`.

Datasets can also be used for validation:

In [38]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32)

model.fit(dataset, epochs=10,
          validation_data=val_dataset)

Train for 32 steps, validate for 4 steps
Epoch 1/10
32/32 [==============================] - 0s 4ms/step - loss: 371637.1958 - categorical_accuracy: 0.0980 - val_loss: 417975.1429 - val_categorical_accuracy: 0.1200
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 352619.4128 - categorical_accuracy: 0.1050 - val_loss: 276694.2941 - val_categorical_accuracy: 0.1100
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 376465.8815 - categorical_accuracy: 0.0980 - val_loss: 326388.9991 - val_categorical_accuracy: 0.0500
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 361749.9208 - categorical_accuracy: 0.1070 - val_loss: 323306.5435 - val_categorical_accuracy: 0.1100
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 414727.1478 - categorical_accuracy: 0.0980 - val_loss: 492189.3892 - val_categorical_accuracy: 0.1100
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 558222.2509 - cat

#### Evaluate and predict
The `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods can use NumPy data and a `tf.data.Dataset`.

Here's how to *evaluate* the inference-mode loss and metrics for the data provided:

In [43]:
# With Numpy arrays
model.evaluate(data, labels, batch_size=32)

# With a Dataset
model.evaluate(dataset)

32/32 [==============================] - 0s 1ms/step - loss: 477809.5419 - categorical_accuracy: 0.1090


[477809.54194641113, 0.109]

And here's how to *predict* the output of the last layer in inference for the data provided, as a NumPy array:

In [44]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


### Build complex models
#### The Functional API
The `tf.keras.Sequential` model is a simple stack of layers that cannot represent arbitrary models. Use the `Keras functional API` to build complex model topologies such as:

- Multi-input models,
- Multi-output models,
- Models with shared layers (the same layer called several times),
- Models with non-sequential data flows (e.g. residual connections).

Building a model with the functional API works like this:

1. A layer instance is callable and returns a tensor.
2. Input tensors and output tensors are used to define a `tf.keras.Model` instance.
3. This model is trained just like the `Sequential` model.

The following example uses the functional API to build a simple, fully-connected network:

In [56]:
inputs = tf.keras.Input(shape=(32,))  # Returns an input placeholder

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

Instantiate the model given inputs and outputs.

In [59]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 260us/sample - loss: 13.6606 - accuracy: 0.1200
Epoch 2/5
1000/1000 [==============================] - 0s 63us/sample - loss: 22.6619 - accuracy: 0.1160
Epoch 3/5
1000/1000 [==============================] - 0s 107us/sample - loss: 36.7885 - accuracy: 0.0990
Epoch 4/5
1000/1000 [==============================] - 0s 129us/sample - loss: 54.1488 - accuracy: 0.1220
Epoch 5/5
1000/1000 [==============================] - 0s 125us/sample - loss: 72.7108 - accuracy: 0.1010


#### Model subclassing
Build a fully-customizable model by subclassing `tf.keras.Model` and defining your own forward pass. Create layers in the `__init__` method and set them as attributes of the class instance. Define the forward pass in the `call` method.

Model subclassing is particularly useful when `eager excution` is enabled, because it allows the forward pass to be written imperatively.

The following example shows a subclassed `tf.keras.Model` using a custom forward pass that does not have to be run imperatively:

In [60]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.dense_1 = layers.Dense(32, activation='relu')
        self.dense_2 = layers.Dense(10, activation='sigmoid')
        
    def call(self, inputs):
        # Define your forward pass here.
        # using layers you previously defined (in '__init__').
        x = self.dense_1(inputs)
        return self.dense_2(x)

Instantiate the new model class:

In [62]:
model = MyModel(num_classes=10)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 242us/sample - loss: 11.5569 - accuracy: 0.0990
Epoch 2/5
1000/1000 [==============================] - 0s 57us/sample - loss: 11.5394 - accuracy: 0.1020
Epoch 3/5
1000/1000 [==============================] - 0s 87us/sample - loss: 11.5337 - accuracy: 0.1130
Epoch 4/5
1000/1000 [==============================] - 0s 96us/sample - loss: 11.5304 - accuracy: 0.1060
Epoch 5/5
1000/1000 [==============================] - 0s 97us/sample - loss: 11.5277 - accuracy: 0.1030


#### Custom layers
Create a custom layer by subclassing `tf.keras.layers.Layer` and implementing the following methods:

- `__init__`: Optionally define sublayers to be used by this layer.
- `build`: Create the weights of the layer. Add weights with the `add_weight` method.
- `call`: Define the forward pass.
- Optionally, a layer can be serialized by implementing the `get_config` method and the `from_config` class method.

Here's an example of a custom layer that implements a `matmul` of an input with a kernel matrix:

In [66]:
class MyLayer(layers.Layer):
    
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return csl(**config)

Create a model using your custom layer:

In [67]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')
])

# The compile step sepcifies the training configuration
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 162us/sample - loss: 11.5598 - accuracy: 0.0950
Epoch 2/5
1000/1000 [==============================] - 0s 39us/sample - loss: 11.5598 - accuracy: 0.0940
Epoch 3/5
1000/1000 [==============================] - 0s 42us/sample - loss: 11.5597 - accuracy: 0.0930
Epoch 4/5
1000/1000 [==============================] - 0s 43us/sample - loss: 11.5592 - accuracy: 0.0980
Epoch 5/5
1000/1000 [==============================] - 0s 53us/sample - loss: 11.5593 - accuracy: 0.1000


In [68]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_layer_2 (MyLayer)         multiple                  320       
_________________________________________________________________
activation_2 (Activation)    multiple                  0         
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


### Callbacks
A callback is an object passed to a model to customize and extend its behavior during training. You can write your own custom callback, or use the built-in `tf.keras.callbacks` that include:

- `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at regular intervals.
- `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning rate.
- `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation performance has stopped improving.
- `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using TensorBoard.

To use a `tf.keras.callbacks.Callback`, pass in to the model's `fit` method:

In [69]:
callbacks = [
    # Interrupt training if 'val_loss' stops improving for over 2 epochs
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    # Write TensorBoard logs to './logs' directory
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 116us/sample - loss: 11.5593 - accuracy: 0.0950 - val_loss: 11.3389 - val_accuracy: 0.0600
Epoch 2/5
1000/1000 [==============================] - 0s 65us/sample - loss: 11.5588 - accuracy: 0.1000 - val_loss: 11.3398 - val_accuracy: 0.0300
Epoch 3/5
1000/1000 [==============================] - 0s 98us/sample - loss: 11.5585 - accuracy: 0.0960 - val_loss: 11.3383 - val_accuracy: 0.0500
Epoch 4/5
1000/1000 [==============================] - 0s 153us/sample - loss: 11.5578 - accuracy: 0.1010 - val_loss: 11.3396 - val_accuracy: 0.0600
Epoch 5/5
1000/1000 [==============================] - 0s 161us/sample - loss: 11.5586 - accuracy: 0.0970 - val_loss: 11.3385 - val_accuracy: 0.0400


### Save and restore
#### Save just the weights values
Save and load the weights of a model using `tf.keras.Model.save_weights`

In [80]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [81]:
# Save weights to a TensorFlow Checkpoint file
model.save_weights('./weights/my_model')

# Restore the model's state,
# This requires a model with the same architecture.
model.load_weights('./weights/my_model')

By default, this saves the model's weights in the `TensorFlow checkpoint` file format. Weights can also be saved to the Keras HDF5 format (the default for the multi-backend implementation of Keras):

In [82]:
! ls ./weights/

checkpoint		      my_model.data-00001-of-00002
my_model.data-00000-of-00002  my_model.index


In [85]:
# Save weights to a HDF5 file
model.save_weights('my_model.h5', save_format='h5')

# Restore the model's state
model.load_weights('my_model.h5')

In [87]:
! ls ./

keras-overview.ipynb  logs  my_model.h5  weights


#### Save just the model configuration
A model's configuration can be saved—this serializes the model architecture without any weights. A saved configuration can recreate and initialize the same model, even without the code that defined the original model. Keras supports JSON and YAML serialization formats:

In [117]:
# Serialize a model to JSON format
json_string = model.to_json()

import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'batch_input_shape': [None, 32],
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': 'float64',
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense_26',
                            

Recreate the model (newly initialized) from the JSON:

In [118]:
fresh_model = tf.keras.models.model_from_json(json_string)

Serializing a model to YAML format requires that you install `pyyaml` before you import TensorFlow:

In [119]:
yaml_string = model.to_yaml()
print(yaml_string)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 32
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float64
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config:
          seed: null
      kernel_regularizer: null
      name: dense_26
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float64
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config:
          seed: null
      kernel_regularizer: null
  

Recreate the model from the YAML:

In [120]:
fresh_model = tf.keras.models.model_from_yaml(yaml_string)

/home/jkpark/.pyenv/versions/3.7.1/envs/tf-gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/saving/model_config.py:76: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


**Caution**: *Subclassed models are not serializable because their architecture is defined by the Python code in the body of the `call` method.*

#### Sasve the entire model in one file
The entire model can be saved to file that contains the weights values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the excat same state—without access to the original code.

In [121]:
# Create a simple model
model = tf.keras.Sequential([
    layers.Dense(10, activation='softmax', input_shape=(32,)),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

# Recreate the exact same model, including weights and optimizer.
model = tf.keras.models.load_model('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 224us/sample - loss: 11.5460 - accuracy: 0.0980
Epoch 2/5
1000/1000 [==============================] - 0s 48us/sample - loss: 11.5473 - accuracy: 0.0970
Epoch 3/5
1000/1000 [==============================] - 0s 78us/sample - loss: 11.5723 - accuracy: 0.0940
Epoch 4/5
1000/1000 [==============================] - 0s 119us/sample - loss: 11.6175 - accuracy: 0.0920
Epoch 5/5
1000/1000 [==============================] - 0s 117us/sample - loss: 11.6525 - accuracy: 0.0950


### Eager execution
`Eager execution` is an impreative programming environment that evaluates operations immediately. This is not required for Keras, but is supported by `tf.keras` and useful for inspecting your program and debugging.

All of the `tf.keras` model-building APIs are compatible with eager execution. And while the `Sequential` and functional APIs can be used, eager execution especially benefits *model subclassing* and building *custom layers*—the APIs that require you to write the forward pass as code (instead of the APIs that create model by assembling existing layers).

### Distribution
#### Multiple GPUs
`tf.keras` models can run on multiple GPUs using `tf.distribute.Strategy`. This API provides distributed training on multiple GPUs with almost no changes to existing code.

Currently, `tf.distribute.MirroredStrategy` is the only supported distribution strategy. `MirroredStrategy` does in-graph replication with synchronous training using all-reduce on a single machine. To use `distribute.Strategys` , nest the optimizer instantiation and model construction and compilation in a `Strategy`'s `.scope()`, then train the model.

The following example distributes a `tf.keras.Model` across multiple GPUs on a single machine.

First, define a model inside the distributed strategy scope:

In [124]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    optimizer = tf.keras.optimizers.SGD(0.2)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


Next, train the model on data as usual:

In [125]:
x = np.random.random((1024, 10))
y = np.random.randint(2, size=(1024, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32)

model.fit(dataset, epochs=1)

Train for 32 steps
32/32 [==============================] - 0s 7ms/step - loss: 0.6994
